In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [2]:
replicate_ids = (0, 1, 2)
prefixes = [f"../workspace/" + str(i) for i in replicate_ids]
for i in replicate_ids:
    os.makedirs(f"{i}", exist_ok=True)

data = {}
for prefix in prefixes:
    data[prefix] = {}

### Trim away our 50ns (dt=10ps) burn-in

In [3]:
def create_trimmed(prefixes):
    for prefix in prefixes:
        u_full = mda.Universe(f"{prefix}/minimized.pdb", f"{prefix}/production.dcd")
        
        with mda.Writer(f"{prefix}/production_trimmed.dcd", u_full.trajectory.n_atoms, dt=u_full.trajectory.time, istart=1) as W:
            for ts in u_full.trajectory[2499:]:
                W.write(u_full.atoms)
        
        u_trimmed = mda.Universe(f"{prefix}/minimized.pdb", f"{prefix}/production_trimmed.dcd")
        data[prefix]["u_trimmed"] = u_trimmed
        
        df = pd.read_csv(f"{prefix}/production.log")
        data[prefix]["df_trimmed"] = df[df["Time (ps)"] >= 50000]
        
    return

In [4]:
def plot_timeseries(prefixes):
    for prefix in prefixes:
        df_trimmed = data[prefix]["df_trimmed"]
        
        fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(9, 9))
        
        colors = {"Kinetic Energy (kJ/mole)": "tab:orange",
                  "Temperature (K)": "tab:red",
                  "Potential Energy (kJ/mole)": "tab:blue",
                  "Total Energy (kJ/mole)": "tab:green",
                 }
        
        for col in ["Kinetic Energy (kJ/mole)", "Temperature (K)"]:
            axes[0].plot(df_trimmed["Time (ps)"]-50_000,
                         df_trimmed[col],
                         color=colors[col],
                         label=col+"\n"+r"$\mu=$"+f"{np.mean(df_trimmed[col]):0.2e}"+r", $\sigma=$" + f"{np.std(df_trimmed[col]):0.2e}")
        axes[0].legend()
        axes[0].grid(axis="x")
        axes[0].set_title(f"Timeseries from {prefix}/production_trimmed.log")
        
        for col in ["Potential Energy (kJ/mole)", "Total Energy (kJ/mole)"]:
            axes[1].plot(df_trimmed["Time (ps)"]-50_000,
                         df_trimmed[col],
                         color=colors[col],
                         label=col+"\n"+r"$\mu=$"+f"{np.mean(df_trimmed[col]):0.2e}"+r", $\sigma=$" + f"{np.std(df_trimmed[col]):0.2e}")
        axes[1].legend()
        axes[1].grid(axis="x")
        
        plt.xlabel("Time (ns)")
        plt.xlim(0, max(df_trimmed["Time (ps)"]-50_000))
        xticks = np.arange(0, 1_000_001, 1e5)
        plt.xticks(xticks, labels=[int(x/1000) for x in xticks])
        plt.tight_layout()
        fig.savefig(f"{prefix[-1]}/timeseries.png")
        plt.close()
        
    return

$$DCC(i,j) = \frac
{\left< \Delta\mathbf{r}_i(t) \cdot \Delta\mathbf{r}_j(t) \right>_t}
{\sqrt{\left< \| \Delta\mathbf{r}_i(t) \|^2 \right>_t} \sqrt{\left< \| \Delta\mathbf{r}_j(t) \|^2 \right>_t}
}$$
$$
\Delta\mathbf{r}_i(t) = \mathbf{r}_i(t) - \left< \mathbf{r}_i(t)\right>_t
$$

In [5]:
def prepare_for_DCC(prefixes):
    for prefix in prefixes:
        
        # Remove overall translations and rotations
        ref = mda.Universe(f"{prefix}/minimized.pdb")
        u_trimmed = data[prefix]["u_trimmed"]
        u_trimmed.trajectory[2499]
        ref.atoms.positions = u_trimmed.atoms.positions.copy()
        
        align.AlignTraj(
            u_trimmed, ref,
            select="protein and name CA",
            in_memory=True,
            match_atoms=True,
        ).run()
        
        ca = u_trimmed.select_atoms("protein and name CA")
        rmsf = rms.RMSF(ca).run()
        rmsf_values = rmsf.rmsf
        core_mask = rmsf_values < 0.8
        core_resids = ca.resids[core_mask]
        core_sel = "protein and name CA and resid " + " ".join(map(str, core_resids))
        
        align.AlignTraj(
            u_trimmed, ref,
            select=core_sel,
            in_memory=True,
            match_atoms=True,
        ).run()
        
        with mda.Writer(f"{prefix}/production_trimmed_static.dcd", u_trimmed.trajectory.n_atoms, dt=u_trimmed.trajectory.time, istart=1) as W:
            for ts in u_trimmed.trajectory[2499:]:
                W.write(u_trimmed.atoms)
        
        # Create an object that stores the 3D position for every Ca for every frame
        ca = u_trimmed.select_atoms("protein and name CA")
        data[prefix]["ca"] = ca
        ca_pos = np.empty((u_trimmed.trajectory.n_frames, ca.n_atoms, 3), dtype=np.float32)
        for i, ts in enumerate(u_trimmed.trajectory):
            ca_pos[i] = ca.positions
        data[prefix]["ca_pos"] = ca_pos
        
    return

In [6]:
def DCC(i, j, ca_pos):
    r_i = ca_pos[:, i, :]
    r_j = ca_pos[:, j, :]

    deltar_i = r_i - r_i.mean(axis=0)
    deltar_j = r_j - r_j.mean(axis=0)

    # per-frame dot products, then time average
    numerator = np.mean(np.sum(deltar_i * deltar_j, axis=1))

    denom_i = np.mean(np.sum(deltar_i * deltar_i, axis=1))
    denom_j = np.mean(np.sum(deltar_j * deltar_j, axis=1))
    denominator = np.sqrt(denom_i * denom_j)

    return numerator / denominator

In [7]:
def create_DCC_matrices(prefixes):
    for prefix in prefixes:
        
        ca = data[prefix]["ca"]
        ca_pos = data[prefix]["ca_pos"]
        
        DCC_matrix = np.zeros((ca.n_atoms, ca.n_atoms))
        for i in tqdm(range(0, len(DCC_matrix))):
            for j in range(0, len(DCC_matrix)):
                DCC_matrix[i,j] = DCC(i, j, ca_pos)
        data[prefix]["DCC_matrix"] = DCC_matrix
        
        avg_dist_matrix = np.zeros((ca.n_atoms, ca.n_atoms))
        for i in tqdm(range(0, len(avg_dist_matrix))):
            for j in range(0, len(avg_dist_matrix)):
                distance = np.linalg.norm(ca_pos[:, i, :] - ca_pos[:, j, :], axis=1)
                avg_dist_matrix[i,j] = np.mean(distance)
        data[prefix]["avg_dist_matrix"] = avg_dist_matrix

    return

In [8]:
def create_individual_heatmaps(prefixes):
    for prefix in prefixes:
        fig = plt.figure(figsize=(6,6))
        ax = plt.axes()
        DCC_matrix = data[prefix]["DCC_matrix"]
        ca = data[prefix]["ca"]
        heatmap = ax.imshow(DCC_matrix, origin='lower', cmap="bwr", vmin=-1.0, vmax=1.0)
        
        cax = fig.add_axes([ax.get_position().x1+0.04,ax.get_position().y0,0.08,ax.get_position().height])
        plt.colorbar(heatmap, cax=cax)
        ax.set_xticks(np.arange(0, ca.n_atoms, 10))
        ax.set_yticks(np.arange(0, ca.n_atoms, 10))
        ax.set_xlabel("Residue index")
        ax.set_ylabel("Residue index")
        ax.set_title("Dynamic cross-correlation matrix\nof $\\alpha$-carbons in apo-PCP1\n" + f"from {prefix}")
        
        ax2 = fig.add_axes([cax.get_position().x1+0.04,cax.get_position().y0,1.2,cax.get_position().height])
        avg_dist_matrix = data[prefix]["avg_dist_matrix"]
        heatmap = ax2.imshow(avg_dist_matrix, origin='lower', cmap="plasma")
        
        cax2 = fig.add_axes([ax2.get_position().x1+0.04,ax2.get_position().y0,0.08,ax2.get_position().height])
        plt.colorbar(heatmap, cax=cax2)
        ax2.set_xticks(np.arange(0, ca.n_atoms, 10))
        ax2.set_yticks(np.arange(0, ca.n_atoms, 10))
        ax2.set_xlabel("Residue index")
        ax2.set_ylabel("Residue index")
        ax2.set_title("Time-average distance ($\\AA$) matrix\nof $\\alpha$-carbons in apo-PCP1" + f"\nfrom {prefix}")

        fig.savefig(f"{prefix[-1]}/heatmap.png", bbox_inches="tight")
        plt.close()

In [9]:
def create_averaged_heatmap(prefixes):

    DCC_matrix = np.mean([data[prefix]["DCC_matrix"] for prefix in prefixes], axis=0)
    np.save("avg_DCC_matrix.npy", DCC_matrix)
    avg_dist_matrix = np.mean([data[prefix]["avg_dist_matrix"] for prefix in prefixes], axis=0)
    np.save("avg_dist_matrix.npy", avg_dist_matrix)
    ca = data[prefixes[0]]["ca"]
    
    fig = plt.figure(figsize=(6,6))
    ax = plt.axes()
    heatmap = ax.imshow(DCC_matrix, origin='lower', cmap="bwr", vmin=-1.0, vmax=1.0)
    
    cax = fig.add_axes([ax.get_position().x1+0.04,ax.get_position().y0,0.08,ax.get_position().height])
    plt.colorbar(heatmap, cax=cax)
    ax.set_xticks(np.arange(0, ca.n_atoms, 10))
    ax.set_yticks(np.arange(0, ca.n_atoms, 10))
    ax.set_xlabel("Residue index")
    ax.set_ylabel("Residue index")
    ax.set_title("Dynamic cross-correlation matrix\nof $\\alpha$-carbons in apo-PCP1\n" + f"averaged from {len(prefixes)} replicates")
    
    ax2 = fig.add_axes([cax.get_position().x1+0.04,cax.get_position().y0,1.2,cax.get_position().height])
    heatmap = ax2.imshow(avg_dist_matrix, origin='lower', cmap="plasma")
    
    cax2 = fig.add_axes([ax2.get_position().x1+0.04,ax2.get_position().y0,0.08,ax2.get_position().height])
    plt.colorbar(heatmap, cax=cax2)
    ax2.set_xticks(np.arange(0, ca.n_atoms, 10))
    ax2.set_yticks(np.arange(0, ca.n_atoms, 10))
    ax2.set_xlabel("Residue index")
    ax2.set_ylabel("Residue index")
    ax2.set_title("Time-average distance ($\\AA$) matrix\nof $\\alpha$-carbons in apo-PCP1" + f"\naveraged from {len(prefixes)} replicates")
    
    fig.savefig("average_heatmap.png", bbox_inches="tight")
    plt.close()

In [10]:
# min_dist = 10
# filtered_DCC_matrix = np.where(avg_dist_matrix >= min_dist, DCC_matrix, np.nan)

# fig = plt.figure(figsize=(6,6))
# ax = plt.axes()
# ax.imshow(np.ones(shape=DCC_matrix.shape)*0.5, cmap="Greys", vmin=0, vmax=1)
# heatmap = ax.imshow(filtered_DCC_matrix, origin='lower', cmap="bwr", vmin=-1.0, vmax=1.0)

# cax = fig.add_axes([ax.get_position().x1+0.04,ax.get_position().y0,0.08,ax.get_position().height])
# plt.colorbar(heatmap, cax=cax)
# ax.set_xticks(np.arange(0, ca.n_atoms, 10))
# ax.set_yticks(np.arange(0, ca.n_atoms, 10))
# ax.set_xlabel("Residue index")
# ax.set_ylabel("Residue index")
# ax.set_title(f"Dynamic cross-correlation matrix\nof $\\alpha$-carbons in apo-PCP1\n{min_dist}$\\AA$ minimum avg. distance threshold")

# plt.show()
# plt.close()

In [11]:
def create_RMSF_hists(prefixes):
    ymax = 0
    for prefix in prefixes:
        ca = data[prefix]["ca"]
        r = rms.RMSF(data[prefix]["ca"]).run().results.rmsf
        if max(r) > ymax:
            ymax = max(r)
        
    for prefix in prefixes:
        ca = data[prefix]["ca"]
        R = rms.RMSF(ca).run()
        fig = plt.figure(figsize=(12,6))
        plt.bar(np.arange(0, ca.n_atoms)+1400, R.results.rmsf)
        plt.xlabel("Residue index")
        plt.xlim(-1+1400, ca.n_atoms+1400)
        plt.ylabel(r"RMSF ($\AA$)")
        plt.ylim(0, ymax+1)
        plt.title(r"Root mean square fluctuations of $\alpha$-carbons" + f"\nfrom {prefix}")
        plt.savefig(f"{prefix[-1]}/RMSF_hist.png")
        plt.close()

In [12]:
def create_averaged_RMSF_hist(prefixes):
    R = [rms.RMSF(data[prefix]["ca"]).run().results.rmsf for prefix in prefixes]
    ca = data[prefixes[0]]["ca"]
    R_avg = np.mean(R, axis=0)
    np.save("R_avg.npy", R_avg, allow_pickle=False)
    R_sem = np.std(R, axis=0) / np.sqrt(len(prefixes))
    fig = plt.figure(figsize=(12,6))
    plt.bar(np.arange(0, ca.n_atoms)+1400, R_avg, yerr=R_sem)
    plt.xlabel("Residue index")
    plt.xlim(-1+1400, ca.n_atoms+1400)
    plt.ylabel(r"RMSF ($\AA$)")
    plt.title(r"Root mean square fluctuations of $\alpha$-carbons" + f"\naveraged from {len(prefixes)} replicates")
    plt.savefig("average_RMSF_hist.png")
    plt.close()

In [13]:
create_trimmed(prefixes)
plot_timeseries(prefixes)
prepare_for_DCC(prefixes)
create_DCC_matrices(prefixes)
create_individual_heatmaps(prefixes)
create_averaged_heatmap(prefixes)
create_RMSF_hists(prefixes)
create_averaged_RMSF_hist(prefixes)

/Users/jakeanderson/miniconda3/envs/openmm/lib/python3.13/site-packages/MDAnalysis/coordinates/DCD.py:171: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
/Users/jakeanderson/miniconda3/envs/openmm/lib/python3.13/site-packages/MDAnalysis/coordinates/DCD.py:171: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
/Users/jakea